# Aandachtsmechanismen en transformers

Een belangrijk nadeel van recurrente netwerken is dat alle woorden in een reeks dezelfde invloed hebben op het resultaat. Dit leidt tot suboptimale prestaties bij standaard LSTM encoder-decoder modellen voor sequentie-tot-sequentie taken, zoals Named Entity Recognition en Machinevertaling. In werkelijkheid hebben specifieke woorden in de invoerreeks vaak meer invloed op de opeenvolgende uitvoer dan andere.

Laten we een sequentie-tot-sequentie model overwegen, zoals machinevertaling. Dit wordt geïmplementeerd door twee recurrente netwerken, waarbij één netwerk (**encoder**) de invoerreeks samenvat in een verborgen toestand, en een ander netwerk, **decoder**, deze verborgen toestand uitrolt naar het vertaalde resultaat. Het probleem met deze aanpak is dat de eindtoestand van het netwerk moeite heeft om het begin van een zin te onthouden, wat leidt tot een slechte kwaliteit van het model bij lange zinnen.

**Aandachtsmechanismen** bieden een manier om het contextuele gewicht van elke invoervector op elke uitvoervoorspelling van het RNN te bepalen. Dit wordt geïmplementeerd door shortcuts te creëren tussen de tussenliggende toestanden van het invoer-RNN en het uitvoer-RNN. Op deze manier houden we bij het genereren van het uitvoersymbool $y_t$ rekening met alle verborgen toestanden van de invoer $h_i$, met verschillende gewichtscoëfficiënten $\alpha_{t,i}$. 

![Afbeelding van een encoder/decoder model met een additieve aandachtlaag](../../../../../translated_images/nl/encoder-decoder-attention.7a726296894fb567.webp)
*Het encoder-decoder model met additief aandachtsmechanisme in [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), geciteerd uit [deze blogpost](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

De aandachtsmatrix $\{\alpha_{i,j}\}$ vertegenwoordigt de mate waarin bepaalde invoerwoorden bijdragen aan de generatie van een specifiek woord in de uitvoerreeks. Hieronder staat een voorbeeld van zo'n matrix:

![Afbeelding van een voorbeelduitlijning gevonden door RNNsearch-50, afkomstig uit Bahdanau - arviz.org](../../../../../translated_images/nl/bahdanau-fig3.09ba2d37f202a6af.webp)

*Afbeelding afkomstig uit [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Aandachtsmechanismen zijn verantwoordelijk voor veel van de huidige of bijna huidige state-of-the-art in natuurlijke taalverwerking. Het toevoegen van aandacht verhoogt echter aanzienlijk het aantal modelparameters, wat leidde tot schaalproblemen met RNN's. Een belangrijke beperking bij het schalen van RNN's is dat de recurrente aard van de modellen het uitdagend maakt om training te batchen en te paralleliseren. In een RNN moet elk element van een reeks in volgorde worden verwerkt, wat betekent dat het niet eenvoudig parallel kan worden uitgevoerd.

De adoptie van aandachtsmechanismen in combinatie met deze beperking leidde tot de creatie van de huidige state-of-the-art Transformer-modellen die we vandaag kennen en gebruiken, van BERT tot OpenGPT3.

## Transformer-modellen

In plaats van de context van elke vorige voorspelling door te geven aan de volgende evaluatiestap, gebruiken **transformer-modellen** **positionele encoderingen** en **aandacht** om de context van een gegeven invoer vast te leggen binnen een opgegeven tekstvenster. De onderstaande afbeelding laat zien hoe positionele encoderingen met aandacht context kunnen vastleggen binnen een bepaald venster.

![Geanimeerde GIF die laat zien hoe de evaluaties worden uitgevoerd in transformer-modellen.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Omdat elke invoerpositie onafhankelijk wordt gemapt naar elke uitvoerpositie, kunnen transformers beter paralleliseren dan RNN's, wat veel grotere en expressievere taalmodellen mogelijk maakt. Elke aandachtshoofd kan worden gebruikt om verschillende relaties tussen woorden te leren, wat de prestaties van downstream-taken in natuurlijke taalverwerking verbetert.

## Een eenvoudig Transformer-model bouwen

Keras bevat geen ingebouwde Transformer-laag, maar we kunnen er zelf een bouwen. Zoals eerder zullen we ons richten op tekstclassificatie van de AG News dataset, maar het is vermeldenswaard dat Transformer-modellen de beste resultaten laten zien bij moeilijkere NLP-taken.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Nieuwe lagen in Keras moeten de `Layer`-klasse subklassen en de `call`-methode implementeren. Laten we beginnen met de **Positional Embedding**-laag. We zullen [wat code uit de officiële Keras-documentatie](https://keras.io/examples/nlp/text_classification_with_transformer/) gebruiken. We gaan ervan uit dat we alle invoersequenties op lengte `maxlen` opvullen.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Deze laag bestaat uit twee `Embedding`-lagen: één voor het embedden van tokens (op de manier die we eerder hebben besproken) en één voor het embedden van tokenposities. Tokenposities worden gecreëerd als een reeks natuurlijke getallen van 0 tot `maxlen` met behulp van `tf.range`, en vervolgens door de embedding-laag gehaald. De twee resulterende embedding-vectoren worden vervolgens bij elkaar opgeteld, wat een positioneel-geëmbedde representatie oplevert van de invoer met de vorm `maxlen`$\times$`embed_dim`.

Laten we nu het transformerblok implementeren. Dit blok zal de uitvoer van de eerder gedefinieerde embedding-laag gebruiken:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Nu zijn we klaar om het complete transformer-model te definiëren:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Modellen

**BERT** (Bidirectional Encoder Representations from Transformers) is een zeer groot meerlagig transformernetwerk met 12 lagen voor *BERT-base* en 24 lagen voor *BERT-large*. Het model wordt eerst voorgetraind op een grote hoeveelheid tekstdata (WikiPedia + boeken) met behulp van ongesuperviseerd leren (voorspellen van gemaskeerde woorden in een zin). Tijdens de voortraining absorbeert het model een aanzienlijk niveau van taalbegrip, wat vervolgens kan worden benut met andere datasets door middel van fine-tuning. Dit proces wordt **transfer learning** genoemd.

![afbeelding van http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/nl/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Er zijn veel variaties van Transformer-architecturen, waaronder BERT, DistilBERT, BigBird, OpenGPT3 en meer, die kunnen worden fijn afgestemd.

Laten we eens kijken hoe we een voorgetraind BERT-model kunnen gebruiken om ons traditionele probleem van sequentieclassificatie op te lossen. We zullen het idee en wat code lenen van de [officiële documentatie](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Om voorgetrainde modellen te laden, zullen we **Tensorflow hub** gebruiken. Laten we eerst de BERT-specifieke vectorizer laden:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Het is belangrijk dat je dezelfde vectorizer gebruikt als degene waarmee het oorspronkelijke netwerk is getraind. Bovendien retourneert de BERT-vectorizer drie componenten:
* `input_word_ids`, wat een reeks tokennummers is voor de invoerzinnen
* `input_mask`, die aangeeft welk deel van de reeks daadwerkelijke invoer bevat en welk deel opvulling is. Dit lijkt op de maskering die wordt geproduceerd door de `Masking`-laag
* `input_type_ids` wordt gebruikt voor taken met taalmodellering en maakt het mogelijk om twee invoerzinnen in één reeks te specificeren.

Vervolgens kunnen we de BERT-feature extractor instantiëren:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Dus, de BERT-laag levert een aantal nuttige resultaten op:
* `pooled_output` is het resultaat van het middelen van alle tokens in de reeks. Je kunt het zien als een intelligente semantische embedding van het hele netwerk. Het is gelijk aan de output van de `GlobalAveragePooling1D`-laag in ons vorige model.
* `sequence_output` is de output van de laatste transformerlaag (komt overeen met de output van `TransformerBlock` in ons model hierboven).
* `encoder_outputs` zijn de outputs van alle transformerlagen. Aangezien we een BERT-model met 4 lagen hebben geladen (zoals je waarschijnlijk kunt afleiden uit de naam, die `4_H` bevat), heeft het 4 tensors. De laatste is hetzelfde als `sequence_output`.

Nu gaan we het end-to-end classificatiemodel definiëren. We zullen gebruik maken van *functionele modeldefinitie*, waarbij we de invoer van het model definiëren en vervolgens een reeks expressies geven om de output te berekenen. We zullen ook de gewichten van het BERT-model niet-trainbaar maken en alleen de uiteindelijke classifier trainen:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Ondanks het feit dat er weinig trainbare parameters zijn, is het proces behoorlijk traag, omdat de BERT feature extractor veel rekenkracht vereist. Het lijkt erop dat we geen redelijke nauwkeurigheid hebben kunnen bereiken, mogelijk door een gebrek aan training of een tekort aan modelparameters.

Laten we proberen de BERT-gewichten te ontgrendelen en deze ook te trainen. Dit vereist een zeer kleine leersnelheid en een meer zorgvuldige trainingsstrategie met **warmup**, waarbij gebruik wordt gemaakt van de **AdamW** optimizer. We zullen het `tf-models-official` pakket gebruiken om de optimizer te creëren:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Zoals je kunt zien, verloopt de training vrij traag - maar je kunt experimenteren en het model een paar epochs (5-10) trainen om te zien of je een beter resultaat kunt behalen in vergelijking met de methoden die we eerder hebben gebruikt.

## Huggingface Transformers-bibliotheek

Een andere veelgebruikte (en iets eenvoudigere) manier om Transformer-modellen te gebruiken is de [HuggingFace-pakket](https://github.com/huggingface/), dat eenvoudige bouwstenen biedt voor verschillende NLP-taken. Het is beschikbaar voor zowel Tensorflow als PyTorch, een ander zeer populair neurale netwerkframework.

> **Opmerking**: Als je niet geïnteresseerd bent in hoe de Transformers-bibliotheek werkt, kun je naar het einde van dit notebook gaan, omdat je niets wezenlijk anders zult zien dan wat we hierboven hebben gedaan. We zullen dezelfde stappen herhalen om een BERT-model te trainen met een andere bibliotheek en een aanzienlijk groter model. Het proces omvat dus een vrij lange training, dus je kunt er ook voor kiezen om alleen door de code te bladeren.

Laten we eens kijken hoe ons probleem kan worden opgelost met behulp van [Huggingface Transformers](http://huggingface.co).


Het eerste wat we moeten doen, is het model kiezen dat we gaan gebruiken. Naast enkele ingebouwde modellen bevat Huggingface een [online modelrepository](https://huggingface.co/models), waar je veel meer vooraf getrainde modellen van de community kunt vinden. Al deze modellen kunnen worden geladen en gebruikt door simpelweg een modelnaam op te geven. Alle benodigde binaire bestanden voor het model worden automatisch gedownload.

Soms moet je je eigen modellen laden. In dat geval kun je de map specificeren die alle relevante bestanden bevat, inclusief parameters voor de tokenizer, het `config.json`-bestand met modelparameters, binaire gewichten, enzovoort.

Vanuit de modelnaam kunnen we zowel het model als de tokenizer instantiëren. Laten we beginnen met een tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Het `tokenizer` object bevat de `encode` functie die direct kan worden gebruikt om tekst te coderen:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

We kunnen ook een tokenizer gebruiken om een reeks te coderen op een manier die geschikt is om aan het model door te geven, d.w.z. inclusief `token_ids`, `input_mask` velden, enz. We kunnen ook specificeren dat we Tensorflow-tensoren willen door het argument `return_tensors='tf'` te geven:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

In ons geval zullen we een vooraf getraind BERT-model gebruiken genaamd `bert-base-uncased`. *Uncased* geeft aan dat het model niet gevoelig is voor hoofdletters en kleine letters.

Bij het trainen van het model moeten we een getokeniseerde reeks als invoer leveren, en daarom zullen we een gegevensverwerkingspipeline ontwerpen. Aangezien `tokenizer.encode` een Python-functie is, zullen we dezelfde aanpak gebruiken als in de vorige eenheid door deze aan te roepen met `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Nu kunnen we het daadwerkelijke model laden met behulp van het `BertForSequenceClassification`-pakket. Dit zorgt ervoor dat ons model al een vereiste architectuur heeft voor classificatie, inclusief de uiteindelijke classifier. Je zult een waarschuwingsbericht zien waarin staat dat de gewichten van de uiteindelijke classifier niet zijn geïnitialiseerd en dat het model pre-training nodig heeft - dat is helemaal prima, want dat is precies wat we gaan doen!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Zoals je kunt zien in `summary()`, bevat het model bijna 110 miljoen parameters! Vermoedelijk, als we een eenvoudige classificatietaak willen uitvoeren op een relatief kleine dataset, willen we de BERT-baselaag niet trainen:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Nu kunnen we beginnen met trainen!

> **Opmerking**: Het trainen van een volledige BERT-model kan erg tijdrovend zijn! Daarom trainen we het alleen voor de eerste 32 batches. Dit is slechts om te laten zien hoe het trainen van het model wordt opgezet. Als je geïnteresseerd bent in volledige training - verwijder gewoon de parameters `steps_per_epoch` en `validation_steps`, en bereid je voor om te wachten!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Als je het aantal iteraties verhoogt, lang genoeg wacht en traint over meerdere epochs, kun je verwachten dat BERT-classificatie ons de beste nauwkeurigheid geeft! Dat komt omdat BERT de structuur van de taal al behoorlijk goed begrijpt, en we alleen de laatste classifier hoeven te fine-tunen. Maar omdat BERT een groot model is, duurt het hele trainingsproces lang en vereist het serieuze rekenkracht! (GPU, en bij voorkeur meer dan één).

> **Opmerking:** In ons voorbeeld hebben we een van de kleinste voorgetrainde BERT-modellen gebruikt. Er zijn grotere modellen die waarschijnlijk betere resultaten opleveren.


## Belangrijkste punten

In deze eenheid hebben we zeer recente modelarchitecturen op basis van **transformers** bekeken. We hebben ze toegepast op onze taak voor tekstclassificatie, maar op dezelfde manier kunnen BERT-modellen worden gebruikt voor entiteitsextractie, vraag-antwoord systemen en andere NLP-taken.

Transformermodellen vertegenwoordigen de huidige state-of-the-art in NLP, en in de meeste gevallen zou dit de eerste oplossing moeten zijn waarmee je begint te experimenteren bij het implementeren van aangepaste NLP-oplossingen. Het begrijpen van de basisprincipes van recurrente neurale netwerken, zoals besproken in deze module, is echter van cruciaal belang als je geavanceerde neurale modellen wilt bouwen.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
